In [4]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 50.965,
	"longitude": 5.5008,
	"start_date": "2020-08-01",
	"end_date": "2020-08-31",
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "shortwave_radiation_instant", "direct_radiation_instant", "diffuse_radiation_instant", "direct_normal_irradiance_instant", "wind_speed_10m", "wind_direction_10m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_rain = hourly.Variables(5).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(8).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(9).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(10).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(11).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(12).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(13).ValuesAsNumpy()
hourly_shortwave_radiation_instant = hourly.Variables(14).ValuesAsNumpy()
hourly_direct_radiation_instant = hourly.Variables(15).ValuesAsNumpy()
hourly_diffuse_radiation_instant = hourly.Variables(16).ValuesAsNumpy()
hourly_direct_normal_irradiance_instant = hourly.Variables(17).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(18).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["shortwave_radiation_instant"] = hourly_shortwave_radiation_instant
hourly_data["diffuse_radiation_instant"] = hourly_diffuse_radiation_instant
hourly_data["direct_normal_irradiance_instant"] = hourly_direct_normal_irradiance_instant
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe = hourly_dataframe.set_index('date')
hourly_dataframe.head()


Coordinates 50.93145751953125°N 5.464285850524902°E
Elevation 65.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,temperature_2m,relative_humidity_2m,pressure_msl,surface_pressure,cloud_cover,shortwave_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,wind_speed_10m,wind_direction_10m
date,,,,,,,,,,
2020-08-01 00:00:00+00:00,25.254499,43.628517,1012.500000,1004.998657,21.000000,0.0,0.0,0.0,14.168641,207.216019
2020-08-01 01:00:00+00:00,24.154501,51.567719,1012.400024,1004.872070,35.700005,0.0,0.0,0.0,9.290511,234.462234
2020-08-01 02:00:00+00:00,23.704500,59.134537,1013.200012,1005.654663,30.600002,0.0,0.0,0.0,14.512064,246.614761
2020-08-01 03:00:00+00:00,22.504499,67.354500,1013.799988,1006.219604,46.800003,0.0,0.0,0.0,13.779114,250.144699
2020-08-01 04:00:00+00:00,21.904501,68.974640,1013.599976,1006.005615,54.600002,0.0,0.0,0.0,9.726665,267.878937


In [2]:
import pickle
hourly_dataframe.to_pickle('openmeteo.pickle')